In [1]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.set_random_seed(777)

# mnist 데이터셋을 호출
mnist = input_data.

# learning_rate
learning_rate = 0.001
# 15번 학습
training_epochs = 15
# img 100개씩 순환하며 학습
batch_size = 100

# dropout rate는 0.7~0.5를 사용하지만 테스트에서는 1을 주어야한다.
keep_prob = tf.placeholder(tf.float32)

# input img 를 담을 placeholder 28*28 = 784
X = tf.placeholder(tf.float32,[None,784])

# 우리가 넣을 이미지의 크기를 정의 28*28*1 (BLACK/WHITE) -1은 몇개가 들어올지 정해지지 않았을 때
X_img = tf.reshape(X,[-1,28,28,1])

# output이 담길 placeholder 숫자의 수 0~9까지10개
Y = tf.placeholder(tf.float32,[None,10])

#L1 image shape = (?,28,28,1) 28*28*1 의 이미지 n개
# filter의 크기는 3*3*1 의 필터 32개를 사용한다고 정의한다. 필터의 크기에서 채널의 크기는 입력과 지켜주자
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))

# Conv -> (?,28,28,32) 패딩 SAME 스트라이드 1이기 때문에 같은 크기가 될 때까지 이동하면서 Conv를 추출 하지만 필터의 수가 32개이기 때문에
# 28*28*32가 된다. 25,088
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
# 활성화함수 relu 적용
L1 = tf.nn.relu(L1)
# Pool -> (?,14,14,32) 6,272
# maxpooling 커널의 크기는 2*2 스트라이드 2 패딩은 SAME으로 절반이 된다.
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],
                   strides = [1,2,2,1], padding = 'SAME')

# overfitting을 방지하기 위한 드랍아웃
L1 = tf.nn.dropout(L1,keep_prob = keep_prob)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
# L2 input image shape = (?,14,14,32)
# filter 64개 사용
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))

# Conv -> (?,14,14,64)
# 위에서 나온 L1이 입력으로 들어감
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')

# L2에 활성화함수 적용
L2 = tf.nn.relu(L2)

# Pool -> (?,7,7,64)
# maxpooling 커널크기 2 스트라이드 2 필터갯수 64-> 7*7*64가 됨
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

In [5]:
# L3 input image shape = (?,7,7,64)
# filter 128개 사용
W3 = tf.Variable(tf.random_normal([7,7,64,128], stddev = 0.01))

# Conv -> (?,7,7,128)
# 위에서 나온 L2이 입력으로 들어감
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')

# L3에 활성화함수 적용
L3 = tf.nn.relu(L3)

# Pool -> (?,4,4,128)
# maxpooling 커널크기 2 스트라이드 2 필터갯수 128 -> 4*4*128이 됨
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

# L3를 하나의 차원으로 펼친다.
L3_flat = tf.reshape(L3, [-1,4*4*128])


In [8]:
# L4 FC 4*4*128 input -> 625 output
W4 = tf.get_variable("W4_1", shape=[4*4*128,625], initializer=tf.contrib.layers.xavier_initializer())

# bias는 output 크기가 같아야함
b4 = tf.Variable(tf.random_normal([625]))

# L3_flat * W4 + b4를 한 뒤에 활성화함수 적용
L4 = tf.nn.relu(tf.matmul(L3_flat,W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [9]:
W5 = tf.get_variable("W5", shape=[625,10], initializer=tf.contrib.layers.xavier_initializer())

b5 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L4, W5) + b5

# loss function과 최적화 함수 정의
# 크로스엔트로피와 소프트맥스를 우리가 세운 가설 logit에 적용하고 그것의 평균을 구한다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels = Y))

# 경사하강법을 통해 loss가 최소가 되는 방향으로 최적화 진행
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
# 텐서플로우 세션 생성
sess = tf.Session()

# 글로벌 변수 초기화
sess.run(tf.global_variables_initializer())

print('학습 시작')
for epoch in range(training_epochs):
    avg_cost = 0
    
    # 학습 이미지 batch_size 만큼 가져오면서 학습
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 드랍아웃 0.7
        feed_dict = {X : batch_xs, Y : batch_ys, keep_prob: 0.7}
        c, _ = sess.run([loss,optimizer],feed_dict = feed_dict)
        avg_cost += c / total_batch
    print('Epoch : ', '%04d' % (epoch +1), 'Cost : ', '{:.9f}'.format(avg_cost))
print('학습 끝')

학습 시작
Epoch :  0001 Cost :  0.316606152
Epoch :  0002 Cost :  0.078444657
Epoch :  0003 Cost :  0.057444553
Epoch :  0004 Cost :  0.046861417
Epoch :  0005 Cost :  0.040452211
Epoch :  0006 Cost :  0.034965851
Epoch :  0007 Cost :  0.031288072
Epoch :  0008 Cost :  0.029549464
Epoch :  0009 Cost :  0.027799732
Epoch :  0010 Cost :  0.025330664
Epoch :  0011 Cost :  0.023159391
Epoch :  0012 Cost :  0.021912025
Epoch :  0013 Cost :  0.020249106
Epoch :  0014 Cost :  0.021010585
Epoch :  0015 Cost :  0.020356772
학습 끝


In [17]:
# 정답률 체크
correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('Accuracy : ', sess.run(accuracy,feed_dict = { X : mnist.test.images, Y : mnist.test.labels, keep_prob:1}))

r = random.randint(0, mnist.test.num_examples - 1)
print("Label : ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print("Prediction : ",sess.run(tf.argmax(logits,1), feed_dict={X : mnist.test.images[r:r+1],keep_prob:1}))

Accuracy :  0.9941
Label :  [1]
Prediction :  [1]
